In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [2]:
df= pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df=df.drop(columns=['RowNumber',	'CustomerId',	'Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode Categorical variables
label_encoder= LabelEncoder()
df['Gender']= label_encoder.fit_transform(df['Gender'])

In [5]:
## One hot Encoder Geography
onehotencoder= OneHotEncoder(handle_unknown='ignore')
geo_encoded= onehotencoder.fit_transform(df[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=onehotencoder.get_feature_names_out(['Geography']))



In [6]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## Combine the one hot encoded columns with the original data
df= pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Split the dataset into independent featues and dependent features
X= df.drop('EstimatedSalary',axis=1)  ## Independent Features
y= df['EstimatedSalary'] ## dependent Features

In [9]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [10]:
## train test split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

## Apply scaler
scaler= StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [11]:
## Save the Encoders and Scaler for Later Use.
with open("onehotencoder_geo.pkl",'wb') as file:
    pickle.dump(onehotencoder,file)
with open("labelencoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder,file)
    
with open("scaler.pkl",'wb') as file:
    pickle.dump(scaler,file)

In [12]:
##ANN Regression Problem Statement
import tensorboard as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Build the ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## First hidden Layer connected with input layer 
    Dense(32,activation='relu'), ## Second Hidden layer
    Dense(1)## Output Layer
])

## Compile the Model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()


d:\Salary_prediction_using_ANN_Project\envn\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
import datetime
import tensorflow
from  tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="regression_logs/fit/"+ datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [15]:
## Set up Earlystopping
early_stopping_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [16]:
## train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100991.4922 - mae: 100991.4922 - val_loss: 98547.4766 - val_mae: 98547.4766
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 99306.7734 - mae: 99306.7734 - val_loss: 97254.5781 - val_mae: 97254.5781
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99099.1875 - mae: 99099.1875 - val_loss: 93932.2422 - val_mae: 93932.2422
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 93772.6562 - mae: 93772.6562 - val_loss: 88304.1797 - val_mae: 88304.1797
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 88511.2891 - mae: 88511.2891 - val_loss: 80746.1562 - val_mae: 80746.1562
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 78924.2031 - mae: 78924.2031 - val_loss: 72341.5625 - val_mae: 72341.5625
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 71500.8438 - mae: 71500.8438 - val_loss: 64443.5508 - val_mae: 64443.5508
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [17]:
%load_ext tensorboard

In [18]:
%tensorboard --logdir regression_logs/fit

In [19]:
## Evaluate the model
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f"Test MAE {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50998.6055 - mae: 50998.6055
Test MAE 50186.18359375


In [20]:
model.save("regression.h5")